<a href="https://colab.research.google.com/github/bhadreshpsavani/LLMOPsNotes/blob/main/AWS_SageMaker_Tracking_For_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AWS Bedrock Chat

In this notebook we will explore, how we can use `BedRockChat` models to implement LLM based project

I will try to produce individual AWS Componet in Each Notebook, This can be used for reference for any implementation

## Installation

In [1]:
!pip install -q langchain-aws langchain-community boto3 python-dotenv sagemaker

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.8/82.8 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.2/139.2 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.5/12.5 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.8/997.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 394.9/394.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.5/150.5 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.7/82.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 82.1/8

### Get AWS Credentials and store it in .env file

We will require `AWS_SECRET_ACCESS_KEY` and `AWS_ACCESS_KEY_ID` from AWS IAM section for accessing AWS bases services using Python SDK

In [2]:
import dotenv
""" We need to store below access credentials in .env file stored in root filder
AWS_SESSION_TOKEN
AWS_SECRET_ACCESS_KEY, AWS_ACCESS_KEY_ID
"""
# load environment variable of aws account
dotenv.load_dotenv()

False

In [4]:
from google.colab import userdata

In [35]:
import boto3
import os
from langchain_community.callbacks.sagemaker_callback import SageMakerCallbackHandler
from langchain.agents import initialize_agent, load_tools
from langchain.chains import LLMChain, SimpleSequentialChain
from langchain_core.prompts import PromptTemplate
from sagemaker.analytics import ExperimentAnalytics
from sagemaker.experiments.run import Run
from sagemaker.session import Session

from langchain_aws import BedrockLLM

In [8]:

# Initialize boto3 client with bedrock-runtime to access bedrock related feature
bedrock_client = boto3.client(
    'bedrock-runtime',
    region_name="us-east-1",
    aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),  # Retrieve from environment variables
    aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY')  # Optional session token
)

## LLM Prompt Tracking

In [24]:
import boto3

session  = boto3.Session(
    region_name="us-east-1",
    aws_access_key_id=userdata.get('AWS_ACCESS_KEY_ID'),  # Retrieve from environment variables
    aws_secret_access_key=userdata.get('AWS_SECRET_ACCESS_KEY')  # Optional session token
    )

In [39]:
# LLM Hyperparameters
HPARAMS = {
    "temperature": 0.1,
}

# Bucket used to save prompt logs (Use `None` is used to save the default bucket or otherwise change it)
BUCKET_NAME = "knowlage-base"

# Experiment name
EXPERIMENT_NAME = "langchain-sagemaker-tracker"

# Create SageMaker Session with the given bucket
sagemaker_session  = Session(
    default_bucket=BUCKET_NAME,
    boto_session=session
    )


In [40]:
RUN_NAME = "run-scenario-1"
PROMPT_TEMPLATE = "tell me a joke about {topic}"
INPUT_VARIABLES = {"topic": "fish"}

In [41]:
with Run(
    experiment_name=EXPERIMENT_NAME, run_name=RUN_NAME, sagemaker_session=sagemaker_session
) as run:
    # Create SageMaker Callback
    sagemaker_callback = SageMakerCallbackHandler(run)

    # Define LLM model with callback
    llm = BedrockLLM(
        model_id="amazon.titan-text-express-v1",
        model_kwargs=dict(temperature=0),
        client=bedrock_client,
        callbacks=[sagemaker_callback],
    )
    # Create prompt template
    prompt = PromptTemplate.from_template(template=PROMPT_TEMPLATE)

    # Create LLM Chain
    chain = LLMChain(llm=llm, prompt=prompt, callbacks=[sagemaker_callback])

    # Run chain
    chain.run(**INPUT_VARIABLES)

    # Reset the callback
    sagemaker_callback.flush_tracker()

## Scenario 2 - Sequential Chain

In [46]:
RUN_NAME = "run-scenario-2"

PROMPT_TEMPLATE_1 = """You are a playwright. Given the title of play, it is your job to write a synopsis for that title.
Title: {title}
Playwright: This is a synopsis for the above play:"""
PROMPT_TEMPLATE_2 = """You are a play critic from the New York Times. Given the synopsis of play, it is your job to write a review for that play.
Play Synopsis: {synopsis}
Review from a New York Times play critic of the above play:"""

INPUT_VARIABLES = {
    "title": "documentary about good video games that push the boundary of game design"
}

In [49]:
with Run(
    experiment_name=EXPERIMENT_NAME, run_name=RUN_NAME, sagemaker_session=sagemaker_session
) as run:
    # Create SageMaker Callback
    sagemaker_callback = SageMakerCallbackHandler(run)

    # Create prompt templates for the chain
    prompt_template1 = PromptTemplate.from_template(template=PROMPT_TEMPLATE_1)
    prompt_template2 = PromptTemplate.from_template(template=PROMPT_TEMPLATE_2)

    # Define LLM model with callback
    llm = BedrockLLM(
        model_id="amazon.titan-text-express-v1",
        model_kwargs=dict(temperature=0),
        client=bedrock_client,
        callbacks=[sagemaker_callback],
    )

    # Create chain1
    chain1 = LLMChain(llm=llm, prompt=prompt_template1, callbacks=[sagemaker_callback])

    # Create chain2
    chain2 = LLMChain(llm=llm, prompt=prompt_template2, callbacks=[sagemaker_callback])

    # Create Sequential chain
    overall_chain = SimpleSequentialChain(
        chains=[chain1, chain2], callbacks=[sagemaker_callback]
    )

    # Run overall sequential chain
    overall_chain.run({"input": INPUT_VARIABLES})

    # Reset the callback
    sagemaker_callback.flush_tracker()

## Scenario 3 - Agent with Tools

In [52]:
RUN_NAME = "run-scenario-3"
PROMPT_TEMPLATE = "Who is the oldest person alive? And what is their current age raised to the power of 1.51?"

In [60]:
with Run(
    experiment_name=EXPERIMENT_NAME, run_name=RUN_NAME, sagemaker_session=sagemaker_session
) as run:
    # Create SageMaker Callback
    sagemaker_callback = SageMakerCallbackHandler(run)

    # Define LLM model with callback
    llm = BedrockLLM(
        model_id="mistral.mistral-small-2402-v1:0",
        model_kwargs=dict(temperature=0),
        client=bedrock_client,
        callbacks=[sagemaker_callback],
    )

    # Define tools
    tools = load_tools([ "llm-math"], llm=llm, callbacks=[sagemaker_callback])

    # Initialize agent with all the tools
    agent = initialize_agent(
        tools, llm, agent="zero-shot-react-description", callbacks=[sagemaker_callback]
    )

    # Run agent
    agent.run(input=PROMPT_TEMPLATE)

    # Reset the callback
    sagemaker_callback.flush_tracker()

### Load Log Data

In [63]:
# Load
logs = ExperimentAnalytics(experiment_name=EXPERIMENT_NAME, sagemaker_session=sagemaker_session)

# Convert as pandas dataframe
df = logs.dataframe(force_refresh=True)

print(df.shape)
df.head()

(3, 58)


,TrialComponentName,DisplayName,agent_action_1 - MediaType,agent_action_1 - Value,agent_action_2 - MediaType,agent_action_2 - Value,agent_finish_1 - MediaType,agent_finish_1 - Value,chain_end_1 - MediaType,chain_end_1 - Value,...,chain_start_2 - MediaType,chain_start_2 - Value,chain_start_3 - MediaType,chain_start_3 - Value,on_text_2 - MediaType,on_text_2 - Value,on_text_3 - MediaType,on_text_3 - Value,on_text_4 - MediaType,on_text_4 - Value
0,langchain-sagemaker-tracker-run-scenario-3,langchain-sagemaker-tracker-run-scenario-3,application/json,s3://knowlage-base/trial-component-artifacts/l...,application/json,s3://knowlage-base/trial-component-artifacts/l...,application/json,s3://knowlage-base/trial-component-artifacts/l...,application/json,s3://knowlage-base/trial-component-artifacts/l...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,langchain-sagemaker-tracker-run-scenario-2,langchain-sagemaker-tracker-run-scenario-2,NaN,NaN,NaN,NaN,NaN,NaN,application/json,s3://knowlage-base/trial-component-artifacts/l...,...,application/json,s3://knowlage-base/trial-component-artifacts/l...,application/json,s3://knowlage-base/trial-component-artifacts/l...,application/json,s3://knowlage-base/trial-component-artifacts/l...,application/json,s3://knowlage-base/trial-component-artifacts/l...,application/json,s3://knowlage-base/trial-component-artifacts/l...
2,langchain-sagemaker-tracker-run-scenario-1,langchain-sagemaker-tracker-run-scenario-1,NaN,NaN,NaN,NaN,NaN,NaN,application/json,s3://knowlage-base/trial-component-artifacts/l...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
